# Gather Data from Various Sources

## ChatGPT

Yes, I know. Model degeneration. Wtvr just let me test my model. ok thx bye.

In [17]:
from bs4 import BeautifulSoup
import pandas as pd

file_path = './raw/chat.html'  # Replace with the path to your HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')

dart_code = []
js_code = []

# Find all <pre> elements with class 
messages = soup.find_all('pre')

for message in messages:
    text = message.text
    if '```' not in text: continue
    # Split the text into code blocks
    code_blocks = text.split('```')
    # get rid of 2 spaces at the beginning of each line
    code_blocks = [code.replace('\n  ', '\n') for code in code_blocks]
    dart_code.append(code_blocks[1][len('dart\n'):])
    js_code.append(code_blocks[3][len('javascript\n'):])

df = pd.DataFrame({'dart': dart_code, 'js': js_code})
df.to_csv('raw/code.csv', index=False)
df.head()


,dart,js
0,"List<int> twoSum(List<int> nums, int target)...","function twoSum(nums, target) {\n const m..."
1,class ListNode {\n int val;\n ListNode...,"function ListNode(val, next = null) {\n t..."
2,int lengthOfLongestSubstring(String s) {\n ...,var lengthOfLongestSubstring = function(s) {...
3,double findMedianSortedArrays(List<int> nums...,"var findMedianSortedArrays = function(nums1,..."
4,String longestPalindrome(String s) {\n if...,var longestPalindrome = function(s) {\n i...


### Load Data

In [18]:
df = pd.read_csv('raw/code.csv')
print(df['dart'][0])

  List<int> twoSum(List<int> nums, int target) {
    Map<int, int> map = {};
    for (int i = 0; i < nums.length; i++) {
      int complement = target - nums[i];
      if (map.containsKey(complement)) {
        return [map[complement]!, i];
      }
      map[nums[i]] = i;
    }
    throw ArgumentError("No two sum solution");
  }
  


## Rosetta Code

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# URL of the Rosetta Code category page for Dart
url = 'https://rosettacode.org/wiki/Category:Dart'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract task links
tasks = soup.find_all('a', href=True, title=True)
task_links = ['https://rosettacode.org' + task['href'] for task in tasks if 'Category' not in task['href'] and str(task['href']).startswith('/wiki/')][5:]

data = []

for link in task_links[:10]:
    print(link)
    task_response = requests.get(link)
    task_soup = BeautifulSoup(task_response.content, 'html.parser')

    # Find Dart and JavaScript code sections
    dart_code = None
    js_code = None

    code_sections = task_soup.find_all('pre')
    for code_section in code_sections:
        code = code_section.text
        print(code)
        print("##################")
        # Check if the immediate previous sibling h2 element contains the text "Dart"
        if code_section.find_previous_sibling('h2') and 'Dart' in code_section.find_previous_sibling('h2').text:
            dart_code = code
            print("Dart\n")
            # print(dart_code)
        # Check if the immediate previous sibling h2 element contains the text "JavaScript"
        if code_section.find_previous_sibling('h2') and 'JavaScript' in code_section.find_previous_sibling('h2').text:
            js_code = code
            print("JavaScript\n")
            # print(js_code)
    break

    if dart_code and js_code:
        data.append({'Task': link.split('/')[-1], 'Dart': dart_code, 'JavaScript': js_code})

# Create DataFrame
df_samples = pd.DataFrame(data)

# Display the DataFrame
print(df_samples)


https://rosettacode.org/wiki/100_doors
V doors = [0B] * 100
L(i) 100
   L(j) (i .< 100).step(i + 1)
      doors[j] = !doors[j]
   print(‘Door ’(i + 1)‘: ’(I doors[i] {‘open’} E ‘close’))
##################
*        100 doors                 13/08/2015
HUNDOOR  CSECT
         USING  HUNDOOR,R12
         LR     R12,R15
         LA     R6,0
         LA     R8,1               step 1
         LA     R9,100
LOOPI    BXH    R6,R8,ELOOPI       do ipass=1 to 100 (R6)
         LR     R7,R6
         SR     R7,R6
         LR     R10,R6             step ipass
         LA     R11,100
LOOPJ    BXH    R7,R10,ELOOPJ      do idoor=ipass to 100 by ipass (R7)
         LA     R5,DOORS-1
         AR     R5,R7
         XI     0(R5),X'01'        doors(idoor)=not(doors(idoor))
NEXTJ    B      LOOPJ
ELOOPJ   B      LOOPI
ELOOPI   LA     R10,BUFFER         R10 address of the buffer
         LA     R5,DOORS           R5 address of doors item
         LA     R6,1               idoor=1 (R6)
         LA     R9,100  

## LeetCode

In [11]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import os

def leetcode_login(driver, username, password):
    driver.get("https://leetcode.com/accounts/login/")
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "id_login"))
    )
    
    driver.find_element(By.ID, "id_login").send_keys(username)
    driver.find_element(By.ID, "id_password").send_keys(password)
    driver.find_element(By.ID, "id_password").send_keys(Keys.RETURN)
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "nav-item"))
    )

def get_problem_links(driver: webdriver.Edge):
    driver.get("https://leetcode.com/problemset/all/")
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "title-cell__ZGos"))
    )
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', class_='title-cell__ZGos')
    return ['https://leetcode.com' + link['href'] for link in links]

def get_solutions(driver, problem_url):
    driver.get(problem_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "css-v3d350"))
    )
    
    # Check for Dart and JavaScript solutions
    dart_code = None
    js_code = None
    
    driver.find_element(By.XPATH, "//a[contains(text(), 'Dart')]").click()
    time.sleep(2)  # Allow time for the page to load Dart solutions
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    dart_code_element = soup.find('div', class_='notranslate')
    if dart_code_element:
        dart_code = dart_code_element.text

    driver.find_element(By.XPATH, "//a[contains(text(), 'JavaScript')]").click()
    time.sleep(2)  # Allow time for the page to load JavaScript solutions
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    js_code_element = soup.find('div', class_='notranslate')
    if js_code_element:
        js_code = js_code_element.text
    
    return dart_code, js_code

def get_leetcode_samples(username, password):
    driver = webdriver.Edge()  # Or the path to your WebDriver
    driver.implicitly_wait(10)
    
    leetcode_login(driver, username, password)
    
    problem_links = get_problem_links(driver)
    data = []
    
    for link in problem_links:
        dart_code, js_code = get_solutions(driver, link)
        if dart_code and js_code:
            data.append({'Problem': link.split('/')[-2], 'Dart': dart_code, 'JavaScript': js_code})
    
    driver.quit()
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Replace these with your LeetCode username and password
username = os.environ.get('GITHUB_USERNAME')
password = os.environ.get('GITHUB_PASSWORD')

# Generate the DataFrame
df_samples = get_leetcode_samples(username, password)

# Display the DataFrame
print(df_samples)

TypeError: 'NoneType' object is not iterable